In [1]:
import sys
sys.path.append("external")
sys.path.append("external/vqgan-taming")
from masked_model import MaskedModel, SequenceModelWrapper
from lowres_trainer import LowResTrainer

from vqgan import VQModel

from masked_transformer import Transformer

import torch
import math

from muse_maskgit_pytorch import TransformerBlocks

import torch.nn as nn

import torch

import math

from vqganconfig import vqgan_config

from masked_attn_mamba import MambaAttn
import sys
sys.path.append("external")
sys.path.append("external/vqgan-taming")
from masked_model import MaskedModel, SequenceModelWrapper
from lowres_trainer import LowResTrainer

from vqgan import VQModel

from masked_transformer import Transformer

import torch
import math

import torch.nn as nn

import torch

import math

from vqganconfig import vqgan_config

def cosine_schedule(t):
    return torch.cos(t * math.pi * 0.5)

vae = VQModel(**vqgan_config)
vae.init_from_ckpt(path = "models/pretrained-vqgan.ckpt")

vae.cuda()

from muse_maskgit_pytorch import MaskGit, MaskGitTransformer

transformer = MaskGitTransformer(
    actual_model = MambaAttn(
      token_size = 2048,
      depth = 18,
      d_state = 16,
    ),
    num_tokens = 16384,       # must be same as codebook size above
    seq_len = 16*16,            # must be equivalent to fmap_size ** 2 in vae
    dim = 2048,                # model dimension
)

base_maskgit = MaskGit(
    transformer = transformer, # transformer
    image_size = 256,          # image size
    cond_drop_prob = 0.25,     # conditional dropout, for classifier free guidance
    self_token_critic = True,
    no_mask_token_prob = 0.25,
).cuda()

/home/ubuntu/miniconda3/envs/muse/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/miniconda3/envs/muse/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ubuntu/miniconda3/envs/muse/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
Restored from  models/pretrained-vqgan.ckpt


In [2]:
vae = VQModel(**vqgan_config)
vae.init_from_ckpt(path = "models/pretrained-vqgan.ckpt")


/home/ubuntu/miniconda3/envs/muse/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ubuntu/miniconda3/envs/muse/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
Restored from  models/pretrained-vqgan.ckpt


In [3]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [4]:
count_parameters(vae)

76737220

In [11]:
input = torch.rand((10, 3, 32, 32))

In [14]:
full_output = vae.encode(input)[2]

In [16]:
for i in range(0, 10):
  partial = vae.encode(input[i].unsqueeze(0))[2]
  print(full_output[i] == partial[0])

[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


tensor([True, True, True, True])
tensor([True, True, True, True])
tensor([True, True, True, True])
tensor([True, True, True, True])
tensor([True, True, True, True])
tensor([True, True, True, True])
tensor([True, True, True, True])
tensor([True, True, True, True])
tensor([True, True, True, True])
tensor([True, True, True, True])
